# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff
using PseudoPotentialData

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice = T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
    atoms     = [ElementPsp(:He, pseudopotentials)]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770823894709                   -0.53    9.0    169ms
  2   -2.772142293067       -2.88       -1.31    1.0    104ms
  3   -2.772170433588       -4.55       -2.55    1.0   99.3ms
  4   -2.772170686287       -6.60       -3.45    1.0    102ms
  5   -2.772170721073       -7.46       -3.89    2.0    121ms
  6   -2.772170722967       -8.72       -5.36    1.0    109ms
  7   -2.772170723012      -10.35       -5.30    3.0    139ms
  8   -2.772170723015      -11.52       -5.91    1.0    481ms
  9   -2.772170723015      -12.57       -6.90    1.0    107ms
 10   -2.772170723015   +  -13.85       -7.84    1.0    129ms
 11   -2.772170723015      -13.56       -7.92    2.0    135ms
 12   -2.772170723015   +  -13.99       -8.70    1.0    107ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.773557997019131

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem. This leads to a density-functional perturbation
theory problem, which is automatically set up and solved in the background.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770502834147                   -0.53    9.0    172ms
  2   -2.772052687405       -2.81       -1.29    1.0   97.9ms
  3   -2.772082921744       -4.52       -2.70    1.0    103ms
  4   -2.772083389667       -6.33       -4.11    1.0    104ms
  5   -2.772083417768       -7.55       -4.82    2.0    120ms
  6   -2.772083417808      -10.40       -5.88    1.0    487ms
  7   -2.772083417811      -11.55       -6.21    2.0    116ms
  8   -2.772083417811      -13.63       -6.35    1.0    104ms
  9   -2.772083417811      -13.73       -7.86    1.0    107ms
 10   -2.772083417811      -13.97       -7.79    3.0    135ms
 11   -2.772083417811      -14.51       -8.26    1.0    108ms
Solving response problem
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      13.0